In [22]:
# Add directory above current directory to path
import sys
sys.path.insert(0, '..')

from sunflower.song_loader import Song,load_from_disk
from sunflower.song_analyzer import SongAnalyzer
from sunflower.utils import export_wav
# Autoreloading for easier development

%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [23]:
# Loading example file 

raw_audio, extension = load_from_disk("../data/guitar_80.wav")

song = Song(raw_audio, extension)

song.print_attributes()

waveform: [[ 4.33444977e-04  2.06470490e-04]
 [ 1.26147270e-03  1.05595589e-03]
 [ 1.95264816e-03  1.75952911e-03]
 ...
 [ 0.00000000e+00 -2.39349902e-07]
 [ 0.00000000e+00 -9.31322575e-10]
 [ 0.00000000e+00  0.00000000e+00]], mono_waveform: [ 3.43552e+05  1.24416e+06  1.99296e+06 ... -1.29000e+02 -1.00000e+00
  0.00000e+00], extension: wav, channels: 2, sr: 44100, sample_width: 4


In [24]:
export_wav(song)